In [1]:
library(tidyverse)
library(mixOmics)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: lattice
Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”
Loaded mixOmics 6.3.2

Thank you for using mixOmics!

How to apply our methods: http://www.mixOmics.org for some examples.
Questions or comments: email us at mixomics[at]math.univ-toulouse.fr  
Any bugs? https://bitbucket.org/klecao/package-mixomics/issues
Cite us:  citation('mixOmics')

Attaching pac

In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
i <- 1

# Load and filter survival data

In [4]:
event_code <- list("Alive" = 0, "Dead" = 1)
covariate_cols <- c("figo_stage", "age_at_diagnosis", "race", "ethnicity")
dep_cols <- c("vital_status", "survival_time")
figo_map_df <- tibble(
    roman_num = c("I", "II", "III", "IV"),
    figo_code = c('1', '2', '3', '4')
)

In [5]:
# survival_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[i], "/survival_data.tsv"))
survival_path <- paste0(dirs$data_dir, "/", unified_dsets[i], "/survival_data.tsv")
survival_df <- load_survival_df(survival_path, event_code)
filtered_survival_df <- survival_df %>%
    dplyr::select(one_of(c("sample_name", dep_cols, covariate_cols))) %>%
    dplyr::filter(rowSums(is.na(.)) == 0, vital_status == 1) %>%
    dplyr::mutate(
        figo_rn = str_extract(figo_stage, "IV|III|II|I")
    ) %>%
    dplyr::inner_join(figo_map_df, by = c("figo_rn" = "roman_num")) %>%
    dplyr::select(-c(figo_rn, figo_stage, vital_status)) %>%
    dplyr::rename(figo_stage = figo_code)

Parsed with column specification:
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)


# Load normalized matrisome count data

In [6]:
norm_matrisome_counts_path <- paste0(dirs$data_dir, "/", unified_dsets[i], "/norm_matrisome_counts.tsv")
norm_survival_counts_df <- read_tsv(norm_matrisome_counts_path) %>%
    dplyr::select(c("geneID", filtered_survival_df$sample_name))
norm_survival_counts_t_df <- norm_survival_counts_df %>%
    column_to_rownames(var = "geneID") %>%
    t() %>%
    as_tibble(rownames = "sample_name")

Parsed with column specification:
cols(
  .default = col_double(),
  geneID = col_character()
)
See spec(...) for full column specifications.


# Join survival and count data

In [7]:
joined_df <- filtered_survival_df %>%
    inner_join(norm_survival_counts_t_df, by = "sample_name")
nrow(joined_df)
head(joined_df)

[1] 66

sample_name,survival_time,age_at_diagnosis,race,ethnicity,figo_stage,PGF,TIMP4,C1QTNF6,TNC,⋯,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TCGA-C5-A1BF-01B-11R-A13Y-07,570,16975,white,not reported,1,10.800637,6.228003,11.669331,13.00293,⋯,9.013453,8.190325,9.503647,14.07799,6.569726,7.315604,4.602649,12.062300,5.649441,16.55841
TCGA-C5-A8YT-01A-11R-A37O-07,633,13253,white,not hispanic or latino,1,7.830611,5.733875,12.445548,13.76547,⋯,9.453187,6.398956,12.288955,13.39633,10.228758,8.542025,4.602649,11.765396,5.318924,13.55632
TCGA-C5-A2LZ-01A-11R-A213-07,3046,24059,white,not hispanic or latino,3,10.135550,4.865349,8.566221,10.84231,⋯,11.492135,7.426145,9.480199,15.92715,6.010764,7.405245,4.602649,7.506731,5.423416,14.73802
TCGA-VS-A9V1-01A-11R-A42T-07,157,17001,white,not reported,4,6.893100,6.334201,10.871571,12.21604,⋯,9.319180,7.444380,8.707254,12.23439,6.011689,10.391491,4.602649,7.767144,5.376786,16.71254
TCGA-C5-A1BE-01B-11R-A13Y-07,2094,23727,white,not reported,1,10.642039,5.348449,8.945220,13.41922,⋯,11.696884,6.384980,9.774029,15.38194,6.205261,7.163053,4.957257,10.113788,5.213815,15.56497
TCGA-C5-A8XH-01A-11R-A37O-07,1394,14444,white,not hispanic or latino,1,9.633752,5.908552,11.672191,13.86377,⋯,11.148165,7.526140,8.561116,14.40442,6.462928,6.102570,4.602649,9.104547,5.088257,15.19735


In [27]:
final_df <- joined_df %>%
    bind_cols(to_one_hot(., "race"), to_one_hot(., "ethnicity"), to_one_hot(., "figo_stage")) %>%
    dplyr::select(-c(race, ethnicity, figo_stage))

In [28]:
X_df = final_df[-c(1, 2)]
y_df = final_df[2]

In [31]:
plsr <- pls(X_df, y_df, ncomp = 2, mode="regression")

Warning message in cor(A[[k]], variates.A[[k]]):
“the standard deviation is zero”

In [37]:
vip <- vip(plsr)